<a href="https://colab.research.google.com/github/big-data-lab-umbc/sea-ice-prediction/blob/main/physics-guided-ML/FLOR_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Initial Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
import numpy as np
from netCDF4 import Dataset
from tempfile import TemporaryFile

## Load Data

In [ ]:
path='/content/drive/MyDrive/UMBC/Polar_Sea_Ice/Mitch Data/'
file=path+'ice_CN_01_01.nc'
df=Dataset(file,'r')

In [ ]:
spear_stats_file=path+'ice.static.nc'
stats_df=Dataset(spear_stats_file,'r')

Reading Data from Variables

In [ ]:
print(df.variables.keys()) # get all variable names
print(stats_df.variables.keys()) # get all variable names

dict_keys(['xt', 'xb', 'yt', 'yb', 'time', 'nv', 'ct', 'CN', 'average_T1', 'average_T2', 'average_DT', 'time_bounds'])
dict_keys(['xt', 'xb', 'yt', 'yb', 'CELL_AREA', 'COSROT', 'GEOLAT', 'GEOLON', 'SINROT'])


### Calculating Weighted Area from SPEARS stats

In [ ]:
stats_df['CELL_AREA'].shape

(200, 360)

In [ ]:
pi = 22/7
r = 6.371e6
area = np.array(stats_df['CELL_AREA'])
weighted_area = area * (4*pi*(r**2))
weighted_area = weighted_area / 10e6
print(weighted_area.shape)

In [ ]:
np.save('1deg_weighted_area.npy',weighted_area)

## FLOR Data Analysis

In [ ]:
for d in df.dimensions.items():
    print(d)

('xt', <class 'netCDF4._netCDF4.Dimension'>: name = 'xt', size = 360)
('xb', <class 'netCDF4._netCDF4.Dimension'>: name = 'xb', size = 361)
('yt', <class 'netCDF4._netCDF4.Dimension'>: name = 'yt', size = 200)
('yb', <class 'netCDF4._netCDF4.Dimension'>: name = 'yb', size = 201)
('time', <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 492)
('nv', <class 'netCDF4._netCDF4.Dimension'>: name = 'nv', size = 2)
('ct', <class 'netCDF4._netCDF4.Dimension'>: name = 'ct', size = 5)


We can also check the dimension size of a variable individually:

In [ ]:
df['CN'].shape

(492, 5, 200, 360)

Reading and printing values inside a variable. Looking for any missing values and filling it with Nans

In [ ]:
ice_data=np.array(df.variables['CN'][:])
fill=df.variables['CN'].missing_value
ice_data[np.where(ice_data==fill)]=np.nan

Sum SIC across all 5 thickness levels

In [ ]:
total_sic = ice_data.sum(axis=1)
print(total_sic.shape)

(492, 200, 360)


## Bias Correction

In [ ]:
model_data #predicted
nsidc_data #ground truth

bias = model_data - nsidc_data
RMSE for both data variables

#interpolate NSIDC data from 25km to 1 degree 
#Ask Mithc for an interpolation method
#Quick search at Google
